In [ ]:
import script as sc

import re
import glob
import math
import csv
import sys
import os
import struct
import json
import numpy as np
import glob
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from obspy import UTCDateTime, Stream, Trace
from tqdm.notebook import tqdm
import seisbench
import seisbench.models as sbm

In [ ]:
Afile_list = glob.glob("*.A*")

In [ ]:
Pfile_list = glob.glob("[0-9]*.P*")

In [ ]:
Afile_list

In [ ]:
Pfile_list

# 記得先在當前目錄建立Pfile_repo資料夾

### 跑多筆資料

In [ ]:
##########
#Parameter
net = ["BB","BH","SMT","OBS","CSMT","BATS","YMS"]
dataset = "original"
model = sc.sbm.EQTransformer.from_pretrained(dataset)
path = ".\\Pfile_repo"
##########
all_diff_arrivaltime_P = {}
all_diff_arrivaltime_S = {}
lost = 0
for ind, ele in enumerate (tqdm(Afile_list)):
    try:
        tmp_Afile = sc.unpackAfile(ele)
        tmp_Pfile = sc.unpackPfile(Pfile_list[ind])
    except:
        lost += 1
        continue
    tmp_Afile_T = sc.RSD_transform(sc.network_filter(tmp_Afile, net))
    tmp_Pdict = sc.predict_dict(tmp_Afile_T, model)
    tmp_PArrivaltime = sc.preds_arrivaltime(tmp_Pdict)
    if tmp_PArrivaltime =={}:
        lost += 1
        continue
    sc.make_P_file(tmp_PArrivaltime, path)
    tmp_diff_arrivaltime_P = sc.diff_arravialtime_P(tmp_PArrivaltime, tmp_Pfile)
    all_diff_arrivaltime_P[ele] = (tmp_diff_arrivaltime_P)
    tmp_diff_arrivaltime_S = sc.diff_arravialtime_S(tmp_PArrivaltime, tmp_Pfile)
    all_diff_arrivaltime_S[ele] = (tmp_diff_arrivaltime_S)

year = tmp_Pfile["ori_time"].year
month = tmp_Pfile["ori_time"].month
day = tmp_Pfile["ori_time"].day
date = f"{year}_{month}_{day}"

with open("all_diff_arrivaltime_P.json", "w") as fp:
    json.dump(all_diff_arrivaltime_P, fp)
with open("all_diff_arrivaltime_S.json", "w") as fp:
    json.dump(all_diff_arrivaltime_S, fp)
with open("info.txt","w") as fp:
    fp.write("Date : " + date)
    fp.write("\n")
    fp.write(f"Numbers_of_event : {len(Afile_list)}")
    fp.write("\n")
    fp.write(f"Dataset : {dataset}")
    fp.write("\n")
    fp.write(f"Model : {model.name}")
    fp.write("\n")
    fp.write(f"available : {len(Afile_list) - lost}")
    fp.write("\n")
    fp.write(f"unavailable : {lost}")

In [ ]:
sc.plot_arrival_P("all_diff_arrivaltime_P.json", "info.txt")
sc.plot_arrival_S("all_diff_arrivaltime_S.json", "info.txt")